In [3]:
import requests, json
import pandas as pd

f = open('config.json','r')
key = json.load(f)
futapikeytest = key['futapikeytest']
futapikeylive = key['futapikeylive']
campeonato_id = 10
urlroot = "https://api.api-futebol.com.br"
head = {'Authorization': 'Bearer ' + futapikeylive}
data = {'app' : 'aaaaa'}

dictpart1 = {
    "Id_Partida": [],
    "Rodada": [],
    "Placar": [],
    "Time_Mandante": [],
    "Time_Visitante": [],
    "Gols_Mandante": [],
    "Gols_Visitante": [],
    "Data_Realizacao": [],
    "Hora_Realizacao": [],
    "Estadio": [],
    "Posse_de_bola_mandante": [], 
    "Finalizacao_mandante": [],
    "Dribles_mandante": [],
    "Posse_de_bola_visitante": [],
    "Finalizacao_visitante": [],
    "Dribles_visitante": [],
}

for rodada in range(26,31): #apenas de 9 em 9 (por dia)

    url = urlroot + f'/v1/campeonatos/{campeonato_id}/rodadas/{rodada}'
    response = requests.post(url, json=data, headers=head)
    rodada = response.json()

    for partida in rodada['partidas']:
        urlpartida = urlroot + f"/v1/partidas/{partida['partida_id']}"
        response = requests.post(urlpartida, json=data, headers=head)
        resposta = response.json()
        
        dictpart1['Id_Partida'].append(resposta['partida_id']) if resposta['partida_id'] else dictpart1['Id_Partida'].append("")
        dictpart1['Rodada'].append(resposta['rodada'])
        dictpart1['Placar'].append(resposta['placar'])
        dictpart1['Time_Mandante'].append(resposta['time_mandante']['nome_popular'])
        dictpart1['Time_Visitante'].append(resposta['time_visitante']['nome_popular'])
        dictpart1['Gols_Mandante'].append(resposta['placar_mandante'])
        dictpart1['Gols_Visitante'].append(resposta['placar_visitante'])
        dictpart1['Data_Realizacao'].append(resposta['data_realizacao'])
        dictpart1['Hora_Realizacao'].append(resposta['hora_realizacao'])
        dictpart1['Estadio'].append(resposta['estadio']["nome_popular"])
        
        if isinstance(resposta["estatisticas"]["mandante"], (dict)):
            dictpart1['Posse_de_bola_mandante'].append(resposta["estatisticas"]["mandante"]["posse_de_bola"])
            dictpart1['Finalizacao_mandante'].append(resposta["estatisticas"]["mandante"]['finalizacao']['total'])
            dictpart1['Dribles_mandante'].append(resposta["estatisticas"]["mandante"]['dribes_bem_sucedidos'])
        else:
            dictpart1['Posse_de_bola_mandante'].append("NaN")
            dictpart1['Finalizacao_mandante'].append("NaN")
            dictpart1['Dribles_mandante'].append("NaN")
       
        if isinstance(resposta["estatisticas"]["visitante"], (dict)): 
            dictpart1['Posse_de_bola_visitante'].append(resposta["estatisticas"]["visitante"]["posse_de_bola"])
            dictpart1['Finalizacao_visitante'].append(resposta["estatisticas"]["visitante"]['finalizacao']['total'])
            dictpart1['Dribles_visitante'].append(resposta["estatisticas"]["visitante"]["posse_de_bola"])
        else:
            dictpart1['Posse_de_bola_visitante'].append("NaN")
            dictpart1['Finalizacao_visitante'].append("NaN")
            dictpart1['Dribles_visitante'].append("NaN")

df = pd.DataFrame(dictpart1)

In [4]:
df.to_excel('rodadas26-30.xlsx', sheet_name="tabestbr2021", index=False)